In [1]:
from sklearn import datasets
import pandas as pd
import numpy as np

iris = datasets.load_iris()

iris_frame = pd.DataFrame(iris.data, columns=iris.feature_names)

iris_frame['target'] = iris.target

# Creating our Pipelines

In [2]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split


class SimplePipeline:
    def __init__(self):
        self.frame = None
        self.X_train, self.X_test, self.y_train, self.Y_test = None, None, None, None
        self.model = None
        self.load_dataset()
    
    def load_dataset(self):
        """Load the dataset and perform train test split."""
        dataset = datasets.load_iris()
        
        # remove units ' (cm)' from variable names
        self.feature_names = [fn[:-5] for fn in dataset.feature_names]
        self.frame = pd.DataFrame(dataset.data, columns=self.feature_names)
        self.frame['target'] = dataset.target
        
        self.X_train, self.X_test, self.y_train, self.y_test = train_test_split(
            self.frame[self.feature_names], self.frame.target, test_size=0.65, random_state=42)
        
    def train(self, algorithm=LogisticRegression):
        
        self.model = algorithm(solver='lbfgs', multi_class='auto')
        self.model.fit(self.X_train, self.y_train)
        
    def predict(self, input_data):
        return self.model.predict(input_data)
        
    def get_accuracy(self):
        
        return self.model.score(X=self.X_test, y=self.y_test)
    
    def run_pipeline(self):
        """Helper method to run multiple pipeline methods with one call."""
        self.load_dataset()
        self.train()

In [3]:
from sklearn.preprocessing import StandardScaler


class PipelineWithDataEngineering(SimplePipeline):
    def __init__(self):
        super().__init__()
        
        self.scaler = StandardScaler()
        
        self.scaler.fit(self.X_train)
    
    def apply_scaler(self):
        self.X_train = self.scaler.transform(self.X_train)
        self.X_test = self.scaler.transform(self.X_test)
        
    def predict(self, input_data):
        scaled_input_data = self.scaler.transform(input_data)
        return self.model.predict(scaled_input_data)
                  
    def run_pipeline(self):
        """Helper method to run multiple pipeline methods with one call."""
        self.load_dataset()
        self.apply_scaler()
        self.train()

# Preparing for the Unit Testing

In [4]:
iris_frame['target'].value_counts()

target
0    50
1    50
2    50
Name: count, dtype: int64

In [10]:
import unittest
from sklearn.metrics import mean_squared_error, accuracy_score

class TestIrisPredictions(unittest.TestCase):
    def setUp(self):
        self.pipeline_v1 = SimplePipeline()
        self.pipeline_v2 = PipelineWithDataEngineering()
        self.pipeline_v1.run_pipeline()
        self.pipeline_v2.run_pipeline()
        
        self.benchmark_predictions = [1.0] * len(self.pipeline_v1.y_test)
    
    def test_accuracy_higher_than_benchmark(self):
        benchmark_accuracy = accuracy_score(
            y_true=self.pipeline_v1.y_test,
            y_pred=self.benchmark_predictions)
        
        predictions = self.pipeline_v1.predict(self.pipeline_v1.X_test)
        
        actual_accuracy = accuracy_score(
            y_true=self.pipeline_v1.y_test,
            y_pred=predictions)
        
        print(f'model accuracy: {actual_accuracy}, benchmark accuracy: {benchmark_accuracy}')
        self.assertTrue(actual_accuracy > benchmark_accuracy)
        
    def test_accuracy_compared_to_previous_version(self):
        v1_accuracy = self.pipeline_v1.get_accuracy()
        v2_accuracy = self.pipeline_v2.get_accuracy()
        print(f'pipeline v1 accuracy: {v1_accuracy}')
        print(f'pipeline v2 accuracy: {v2_accuracy}')
        
        self.assertTrue(v2_accuracy >= v1_accuracy)

In [12]:
import sys

suite = unittest.TestLoader().loadTestsFromTestCase(TestIrisPredictions)
unittest.TextTestRunner(verbosity=1, stream=sys.stderr).run(suite)

F.
FAIL: test_accuracy_compared_to_previous_version (__main__.TestIrisPredictions)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "/var/folders/5w/ykddgrfn7sx_lxz9ztw3m9nm0000gn/T/ipykernel_88028/1509151972.py", line 33, in test_accuracy_compared_to_previous_version
    self.assertTrue(v2_accuracy >= v1_accuracy)
AssertionError: False is not true

----------------------------------------------------------------------
Ran 2 tests in 0.100s

FAILED (failures=1)


pipeline v1 accuracy: 0.9693877551020408
pipeline v2 accuracy: 0.9591836734693877
model accuracy: 0.9693877551020408, benchmark accuracy: 0.32653061224489793


<unittest.runner.TextTestResult run=2 errors=0 failures=1>